## Un esempio di interazione tra agenti

Luca Mari, settembre 2024  

Quest'opera è distribuita con <a href="http://creativecommons.org/licenses/by-nc-sa/4.0" target="_blank">Licenza Creative Commons Attribuzione - Non commerciale - Condividi allo stesso modo 4.0 Internazionale</a>.  
<img src="https://creativecommons.it/chapterIT/wp-content/uploads/2021/01/by-nc-sa.eu_.png" width="100">

**Obiettivo**: comprendere qualche aspetto della logica delle architetture ad agenti.  
**Precompetenze**: basi di Python.

> Per eseguire questo notebook, supponiamo con VSCode, occorre:
> * installare un interprete Python
> * scaricare da https://ollama.com e installare Ollama
> * scaricare da Ollama un modello capace di operare con strumenti, supporremo `llama3.1:8b`:  
>       `ollama pull llama3.1`
> * scaricare da https://code.visualstudio.com/download e installare VSCode
> * eseguire VSCode e attivare le estensioni per Python e Jupyter
> * ancora in VSCode:
>     * creare una cartella di lavoro e renderla la cartella corrente
>     * copiare nella cartella il file di questa attività: [chattingagents.ipynb](chattingagents.ipynb)
>     * aprire il notebook `chattingagents.ipynb`
>     * creare un ambiente virtuale locale Python (Select Kernel | Python Environments | Create Python Environment | Venv, e scegliere un interprete Python):
>     * installare i moduli Python richiesti, eseguendo dal terminale:  
>         `pip install pyautogen`
> * eseguire dalla linea di comando:  
>       `OLLAMA_MODELS=xxx OLLAMA_HOST=127.0.0.1:1234 ollama serve`  
> dove `xxx` è la directory che contiene i modelli Ollama (in Linux potrebbe essere `/var/lib/ollama/.ollama/models`)

Importiamo i moduli Python necessari e specifichiamo la configurazione per il modello linguistico che sarà usato e l'indirizzo del server su cui sarà in esecuzione in locale (per esempio Ollama o LM Studio).

In [2]:
from autogen import UserProxyAgent, ConversableAgent, GroupChat, GroupChatManager

llm_config = {
    "config_list": [{ "base_url":"http://localhost:1234/v1",
                      "model":"llama3.1:8b",
                      "api_key":"not_used" }],
    "timeout": 120,
    "cache_seed": None,
}

Definiamo una semplice architettura con un agente di interfaccia (`manager`), e due altri agenti con compiti assegnati (`creative` e `analyst`).

In [3]:
manager = UserProxyAgent(
    name="interfaccia con l'utente e manager",
    is_termination_msg=(lambda msg: "conclus" in msg["content"].lower()), # a volte potrebbe essere "concluso" o "conclusa"...
    human_input_mode="NEVER",
    code_execution_config=False
)

creative = ConversableAgent(
    name="creativo",
    llm_config=llm_config,
    human_input_mode="NEVER",
    code_execution_config=False,
    system_message="""
    Sei il creativo dell'organizzazione, e il tuo compito è di proporre molteplici idee innovative,
    che il tuo collega analista valuterà e da cui sceglierà quella che ritiene la migliore.
    Per chiarezza, presenta sempre le tue proposte in un elenco.
    """)

analyst = ConversableAgent(
    name="analista",
    llm_config=llm_config,
    human_input_mode="NEVER",
    code_execution_config=False,
    system_message="""
    Sei l'analista dell'organizzazione, e il tuo compito è di valutare le proposte
    del tuo collega creativo, scegliere quella che ritieni la migliore,
    eventualmente modificarla per migliorarla, e motivare la tua scelta.
    Quando ritieni che il risultato raggiunto sia soddisfacente, scrivi 'CONCLUSO'.
    """)

Stabiliamo le regole con cui gli agenti interagiranno tra loro.

In [ ]:
def state_transition(last_speaker, groupchat):
    if last_speaker is manager:
        return creative
    elif last_speaker is creative:
        return analyst
    elif last_speaker is analyst:
        return creative
    else:
        raise ValueError("Invalid last_speaker")

groupchat = GroupChat(
    agents=[manager, creative, analyst],
    messages=[],
    max_round=2,
    speaker_selection_method=state_transition
)

manager = GroupChatManager(groupchat=groupchat)

E infine attiviamo il dialogo tra gli agenti.

In [13]:
manager.initiate_chat(
    creative,
    message="""
    Buongiorno cari colleghi. Dobbiamo scegliere un nome per il corso di introduzione all'intelligenza artificiale
    che stiamo finendo di progettare e che, vi ricordo, è rivolto a manager di organizzazioni che,
    senza voler diventare esperti di questioni tecniche, sono interessati a comprendere 
    potenzialità e rischi di questa tecnologia. Chiedo al creativo di proporre 5 idee e di sottoporle all'analista,
    che farà le sue valutazioni e mi farà di conseguenza la sua proposta, concludendo così il nostro lavoro.
    """)

chat_manager (to creativo):


    Buongiorno cari colleghi. Dobbiamo scegliere un nome per il corso di introduzione all'intelligenza artificiale
    che stiamo finendo di progettare e che, vi ricordo, è rivolto a manager di organizzazioni che,
    senza voler diventare esperti di questioni tecniche, sono interessati a comprendere 
    potenzialità e rischi di questa tecnologia. Chiedo al creativo di proporre 5 idee e di sottoporle all'analista,
    che farà le sue valutazioni e mi farà di conseguenza la sua proposta, concludendo così il nostro lavoro.
    

--------------------------------------------------------------------------------
[autogen.oai.client: 09-23 14:56:54] {349} WARNING - Model llama3.1:8b is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
creativo (to chat_manager):

Ciao a tutti! Sono felice di affrontare questo compito. Ecco le mie 5 idee innovative per un corso di

ChatResult(chat_id=None, chat_history=[{'content': "\n    Buongiorno cari colleghi. Dobbiamo scegliere un nome per il corso di introduzione all'intelligenza artificiale\n    che stiamo finendo di progettare e che, vi ricordo, è rivolto a manager di organizzazioni che,\n    senza voler diventare esperti di questioni tecniche, sono interessati a comprendere \n    potenzialità e rischi di questa tecnologia. Chiedo al creativo di proporre 5 idee e di sottoporle all'analista,\n    che farà le sue valutazioni e mi farà di conseguenza la sua proposta, concludendo così il nostro lavoro.\n    ", 'role': 'assistant', 'name': 'chat_manager'}, {'content': 'Ciao a tutti! Sono felice di affrontare questo compito. Ecco le mie 5 idee innovative per un corso di introduzione all\'intelligenza artificiale (IA) rivolto ai manager:\n\n**Ideas per il nome del corso:**\n\n1. **"L\'Uomo e l\'MACchina" - La trasformazione digitale nell\'organizzazione**: è un titolo che rimanda alle possibili applicazioni dell